## 목차
### 1. Library Import
### 2. 학습데이터 생성
### 3. Light-gbm 모델 훈련
### 4. 교차 검증 점수 확인
### 5. 제출 파일 생성

# 1. Library Import

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')

# 필요한 함수 정의
def make_datetime(x):
    # string 타입의 Time column을 datetime 타입으로 변경
    x     = str(x)
    year  = int(x[:4])
    month = int(x[4:6])
    day   = int(x[6:8])
    hour  = int(x[8:10])
    #mim  = int(x[10:12])
    #sec  = int(x[12:])
    return dt.datetime(year, month, day, hour)

def string2num(x):
    # (,)( )과 같은 불필요한 데이터 정제
    x = re.sub(r"[^0-9]+", '', str(x))
    if x =='':
        return 0
    else:
        return int(x)


PATH = '/content/drive/MyDrive/Colab Notebooks/데이콘_lg/'

# 2. 학습 데이터 생성

## 2.1 train_err

* train_err_data.csv 파일의 errtype column의 value_counts를 카운트하여 학습에 사용.  
* 주어진 기간동안 user_id별로 peroblem가 한 번이라도 발생했는 지를 확인.  

In [ ]:
train_err  = pd.read_csv(PATH+'train_err_data.csv')
display(train_err.head())

user_id            time model_nm       fwver  errtype errcode
0    10000  20201101025616  model_3  05.15.2138       15       1
1    10000  20201101030309  model_3  05.15.2138       12       1
2    10000  20201101030309  model_3  05.15.2138       11       1
3    10000  20201101050514  model_3  05.15.2138       16       1
4    10000  20201101050515  model_3  05.15.2138        4       0

In [ ]:
# 데이터 설명을 확인하면
# ueser_id가 10000부터 24999까지 총 15000개가 연속적으로 존재.
display(train_err.head())
train_user_id_max = 24999
train_user_id_min = 10000
train_user_number = 15000

user_id            time model_nm       fwver  errtype errcode
0    10000  20201101025616  model_3  05.15.2138       15       1
1    10000  20201101030309  model_3  05.15.2138       12       1
2    10000  20201101030309  model_3  05.15.2138       11       1
3    10000  20201101050514  model_3  05.15.2138       16       1
4    10000  20201101050515  model_3  05.15.2138        4       0

In [ ]:
print(np.sort(train_err.errtype.unique()))
# errtype이 1부터 42까지 29를 제외한 41개가 존재.

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 30 31 32 33 34 35 36 37 38 39 40 41 42]


In [ ]:
# errtype_pick = [16, 26, 31, 40, 15, 5, 23, 14, 22, 34, 4, 12, 17, 42, 11, 33]

In [ ]:
# train_err['errtype_pick'] = train_err['errtype'].isin(errtype_pick).astype('int64')

In [ ]:
# train_err = train_err[train_err.errtype_pick == 1]
# train_err

In [ ]:
# user_id와 errtype만을 사용하여 데이터 셋 생성
# 모든 일자에 대해서 errtype별 발생 건수를 count
# pandas의 groupby를 활용할 경우 큰 연산 자원이 필요.
# numpy로 placeholder를 만들어 구현함.
id_error = train_err[['user_id','errtype']].values
error = np.zeros((train_user_number,42))

for person_idx, err in tqdm(id_error):
    # person_idx - train_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    error[person_idx - train_user_id_min,err - 1] += 1
error.shape

100%|██████████| 16554663/16554663 [00:48<00:00, 344314.85it/s]


(15000, 42)

In [ ]:
train_err.dropna(axis = 0, inplace=True)
train_err.shape

(16554662, 6)

In [ ]:
items = train_err.errcode.unique().astype(str)
items

array(['1', '0', '2', ..., '4526', '3965', '25999'], dtype='<U39')

In [ ]:
len(train_err.errcode.unique())

2805

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(items)
labels = encoder.transform(train_err.errcode)
labels.shape

(16554662,)

In [ ]:
train_err['errcode_label'] = labels

In [ ]:
errcode_pick = [2, 4, 279, 2728, 196, 2796, 2755, 
                2799, 2803, 2771, 2517, 1209, 2500, 176, 
                2798, 2518, 2804, 2768, 2791, 297, 2802, 
                2492, 2776, 2785, 2800, 2598, 2579, 2744, 
                2763, 2639, 2774, 2758, 2729, 1537, 2153, 2676]

In [ ]:
train_err['errcode_pick'] = train_err['errcode_label'].isin(errcode_pick).astype('int64')

In [ ]:
train_err.errcode_pick.value_counts()

1    16217812
0      336850
Name: errcode_pick, dtype: int64

In [ ]:
train_err = train_err[train_err.errcode_pick == 1]
train_err

user_id            time model_nm  ... errcode  errcode_label errcode_pick
0           10000  20201101025616  model_3  ...       1              4            1
1           10000  20201101030309  model_3  ...       1              4            1
2           10000  20201101030309  model_3  ...       1              4            1
3           10000  20201101050514  model_3  ...       1              4            1
4           10000  20201101050515  model_3  ...       0              2            1
...           ...             ...      ...  ...     ...            ...          ...
16554658    24999  20201130163051  model_3  ...       1              4            1
16554659    24999  20201130172625  model_3  ...       1              4            1
16554660    24999  20201130172625  model_3  ...       0              2            1
16554661    24999  20201130172631  model_3  ...       0              2            1
16554662    24999  20201130210625  model_3  ...       1              4            1

[16217812 rows x 8 columns]

In [ ]:
train_err.errcode_label.unique()

array([   4,    2,  279, 2755, 2728,  196, 1209,  176, 2517, 2803, 2796,
       2768, 2799, 2771, 2804, 2798, 2518, 2500, 2598, 2579, 2800, 2492,
       2785, 2639, 2791, 2774, 2676, 2776, 2153, 1537, 2729,  297, 2744,
       2802, 2758, 2763])

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder2 = LabelEncoder()
encoder2.fit(train_err.errcode_label.unique())
labels = encoder2.transform(train_err.errcode_label)
labels.shape

(16217812,)

In [ ]:
train_err['errcode_label_2'] = labels

In [ ]:
train_err.errcode_label_2.value_counts().sort_index()

0     2599123
1     8906967
2       12040
3      263577
4      166382
5         982
6       20249
7        7335
8        7392
9        5028
10     332422
11       9317
12     334018
13     129876
14       4023
15      65992
16      23452
17     897863
18        172
19        163
20     104138
21        105
22        157
23      11959
24      34631
25        138
26       1583
27       3914
28       1740
29     219238
30     100940
31    1835600
32       4011
33        392
34     110370
35       2523
Name: errcode_label_2, dtype: int64

In [ ]:
len(train_err.errcode_label_2.unique())

36

In [ ]:
train_err['code_fwver'] = train_err["fwver"].map(str) + " " + train_err["errcode_label_2"].map(str)

In [ ]:
train_err

user_id            time  ... errcode_label_2    code_fwver
0           10000  20201101025616  ...               1  05.15.2138 1
1           10000  20201101030309  ...               1  05.15.2138 1
2           10000  20201101030309  ...               1  05.15.2138 1
3           10000  20201101050514  ...               1  05.15.2138 1
4           10000  20201101050515  ...               0  05.15.2138 0
...           ...             ...  ...             ...           ...
16554658    24999  20201130163051  ...               1  05.15.2138 1
16554659    24999  20201130172625  ...               1  05.15.2138 1
16554660    24999  20201130172625  ...               0  05.15.2138 0
16554661    24999  20201130172631  ...               0  05.15.2138 0
16554662    24999  20201130210625  ...               1  05.15.2138 1

[16217812 rows x 10 columns]

In [ ]:
encoder_fw = LabelEncoder()
encoder_fw.fit(train_err.code_fwver.unique())
labels = encoder_fw.transform(train_err.code_fwver)
labels.shape

(16217812,)

In [ ]:
train_err['code_fwver_label'] = labels

In [ ]:
train_err.code_fwver_label.value_counts().sort_index()

0        3
1       24
2        1
3        1
4      288
      ... 
423    133
424     10
425    209
426      1
427      1
Name: code_fwver_label, Length: 428, dtype: int64

In [ ]:
len(train_err.code_fwver_label.unique())

428

In [ ]:
id_code_fwver = train_err[['user_id','code_fwver_label']].values
code_fwver = np.zeros((train_user_number,428))

for person_idx, code_fw in tqdm(id_code_fwver):
    # person_idx - train_user_id_min 위치에 person_idx, errcode에 해당하는 error값을 +1
    code_fwver[person_idx - train_user_id_min,code_fw] += 1
code_fwver.shape

100%|██████████| 16217812/16217812 [00:44<00:00, 367211.77it/s]


(15000, 428)

In [ ]:
type_code_fw = np.append(error, code_fwver, axis=1)
type_code_fw.shape

(15000, 470)

In [ ]:
# encoder_fw = LabelEncoder()
# encoder_fw.fit(train_err.fwver.unique())
# labels = encoder_fw.transform(train_err.fwver)
# labels.shape

In [ ]:
# train_err['fwver_label'] = labels

In [ ]:
# train_err.fwver_label.value_counts().sort_index()

In [ ]:
# len(train_err.fwver_label.unique())

In [ ]:
# id_fwver = train_err[['user_id','fwver_label']].values
# fwver = np.zeros((train_user_number,37))

# for person_idx, fw in tqdm(id_fwver):
#     # person_idx - train_user_id_min 위치에 person_idx, errcode에 해당하는 error값을 +1
#     fwver[person_idx - train_user_id_min,fw] += 1
# fwver.shape

In [ ]:
# type_code_fwver = np.append(type_code, fwver, axis=1)
# type_code_fwver.shape

In [ ]:
#참고) 일별 합산 데이터 생성 코드      
'''
train_err             = pd.read_csv(PATH+'train_err_data.csv')
train_err['datetime'] = train_err['time'].apply(make_datetime)

min_day = train_err.datetime.min().date()

train_err['days'] = train_err.loc[:,'datetime'].dt.date - min_day
train_err['days'] = train_err['days'].dt.days
display(train_err)

id_error = train_err[['user_id','errtype','days']].values
error = np.zeros((train_user_number,42, 33))
for idx, err, days in tqdm(id_error):
    error[idx - train_user_id_min,err - 1, days] += 1
error.shape
'''

"\ntrain_err             = pd.read_csv(PATH+'train_err_data.csv')\ntrain_err['datetime'] = train_err['time'].apply(make_datetime)\n\nmin_day = train_err.datetime.min().date()\n\ntrain_err['days'] = train_err.loc[:,'datetime'].dt.date - min_day\ntrain_err['days'] = train_err['days'].dt.days\ndisplay(train_err)\n\nid_error = train_err[['user_id','errtype','days']].values\nerror = np.zeros((train_user_number,42, 33))\nfor idx, err, days in tqdm(id_error):\n    error[idx - train_user_id_min,err - 1, days] += 1\nerror.shape\n"

## 2.2 problem

In [ ]:
train_prob = pd.read_csv(PATH+'train_problem_data.csv')
problem = np.zeros(15000)
# error와 동일한 방법으로 person_idx - 10000 위치에 
# person_idx의 problem이 한 번이라도 발생했다면 1
# 없다면 0
problem[train_prob.user_id.unique()-10000] = 1 
problem.shape

(15000,)

# 3. Light-gbm 모델 훈련

In [ ]:
# 변수 이름 변경
# error  -> train_x
# problem-> train_y

train_x = type_code_fw
train_y = problem
del type_code_fw, problem
print(train_x.shape)
print(train_y.shape)

(15000, 470)
(15000,)


In [ ]:
# Train
#-------------------------------------------------------------------------------------
# validation auc score를 확인하기 위해 정의
def f_pr_auc(probas_pred, y_true):
    labels=y_true.get_label()
    p, r, _ = precision_recall_curve(labels, probas_pred)
    score=auc(r,p) 
    return "pr_auc", score, True
#-------------------------------------------------------------------------------------
models     = []
recalls    = []
precisions = []
auc_scores   = []
threshold = 0.5
# 파라미터 설정
params =      {
                'boosting_type' : 'gbdt',
                'objective'     : 'binary',
                'metric'        : 'auc',
                'seed': 1015
                }
#-------------------------------------------------------------------------------------
# 5 Kfold cross validation
k_fold = KFold(n_splits=5, shuffle=True, random_state=42)
for train_idx, val_idx in k_fold.split(train_x):

    # split train, validation set
    X = train_x[train_idx]
    y = train_y[train_idx]
    valid_x = train_x[val_idx]
    valid_y = train_y[val_idx]

    d_train= lgb.Dataset(X, y)
    d_val  = lgb.Dataset(valid_x, valid_y)
    
    #run traning
    model = lgb.train(
                        params,
                        train_set       = d_train,
                        num_boost_round = 2000,
                        valid_sets      = d_val,
                        feval           = f_pr_auc,
                        verbose_eval    = 20, 
                        early_stopping_rounds = 100
                       )
    
    # cal valid prediction
    valid_prob = model.predict(valid_x)
    valid_pred = np.where(valid_prob > threshold, 1, 0)
    
    # cal scores
    recall    = recall_score(    valid_y, valid_pred)
    precision = precision_score( valid_y, valid_pred)
    auc_score = roc_auc_score(   valid_y, valid_prob)

    # append scores
    models.append(model)
    recalls.append(recall)
    precisions.append(precision)
    auc_scores.append(auc_score)

    print('==========================================================')

Training until validation scores don't improve for 100 rounds.
[20]	valid_0's auc: 0.805671	valid_0's pr_auc: 0.801864
[40]	valid_0's auc: 0.813751	valid_0's pr_auc: 0.815472
[60]	valid_0's auc: 0.814247	valid_0's pr_auc: 0.815635
[80]	valid_0's auc: 0.813159	valid_0's pr_auc: 0.814951
[100]	valid_0's auc: 0.811455	valid_0's pr_auc: 0.814396
[120]	valid_0's auc: 0.809861	valid_0's pr_auc: 0.813286
[140]	valid_0's auc: 0.809267	valid_0's pr_auc: 0.812253
[160]	valid_0's auc: 0.807709	valid_0's pr_auc: 0.811372
Early stopping, best iteration is:
[69]	valid_0's auc: 0.815438	valid_0's pr_auc: 0.816689
Training until validation scores don't improve for 100 rounds.
[20]	valid_0's auc: 0.815293	valid_0's pr_auc: 0.335998
[40]	valid_0's auc: 0.82004	valid_0's pr_auc: 0.38609
[60]	valid_0's auc: 0.82008	valid_0's pr_auc: 0.394919
[80]	valid_0's auc: 0.817504	valid_0's pr_auc: 0.397534
[100]	valid_0's auc: 0.817818	valid_0's pr_auc: 0.393342
[120]	valid_0's auc: 0.817827	valid_0's pr_auc: 0.386

# 4. 교차검증 점수 확인

In [ ]:
print(np.mean(auc_scores))

0.8130494803097846


# 5. 제출 파일 생성

In [ ]:
test_err  = pd.read_csv(PATH+'test_err_data.csv')
display(test_err.head())

user_id            time model_nm       fwver  errtype errcode
0    30000  20201101030227  model_1  04.16.3553       31       1
1    30000  20201101030227  model_1  04.16.3553       33       2
2    30000  20201101030228  model_1  04.16.3553       15       1
3    30000  20201101030256  model_1  04.16.3553       22       1
4    30000  20201101030300  model_1  04.16.3553       11       1

In [ ]:
# 데이터 설명을 확인하면
# test 데이터는 ueser_id가 30000부터 44998까지 총 14999개가 존재.
test_user_id_max = 44998
test_user_id_min = 30000
test_user_number = 14999

In [ ]:
test_err.dropna(axis = 0, inplace=True)
test_err.shape

(16532644, 6)

In [ ]:
id_error = test_err[['user_id','errtype']].values
test_x = np.zeros((test_user_number,42))
for person_idx, err in tqdm(id_error):
    # person_idx - test_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    test_x[person_idx - test_user_id_min,err - 1] += 1
test_x = test_x.reshape(test_x.shape[0],-1)
print(test_x.shape)

100%|██████████| 16532644/16532644 [00:49<00:00, 336089.28it/s]

(14999, 42)


In [ ]:
test_err

user_id            time model_nm       fwver  errtype errcode
0           30000  20201101030227  model_1  04.16.3553       31       1
1           30000  20201101030227  model_1  04.16.3553       33       2
2           30000  20201101030228  model_1  04.16.3553       15       1
3           30000  20201101030256  model_1  04.16.3553       22       1
4           30000  20201101030300  model_1  04.16.3553       11       1
...           ...             ...      ...         ...      ...     ...
16532643    44998  20201130210050  model_1  04.16.3553       40       0
16532644    44998  20201130211831  model_1  04.16.3553       31       1
16532645    44998  20201130211832  model_1  04.16.3553       15       1
16532646    44998  20201130212259  model_1  04.16.3553       16       1
16532647    44998  20201130212316  model_1  04.16.3553       31       0

[16532644 rows x 6 columns]

In [ ]:
a = set(test_err.errcode).intersection(train_err.errcode)
print(a)

{'connection fail to establish', 'NFANDROID2', '4', 'UNKNOWN', '90', '95', 'B-A8002', 'H-51046', 'PHONE_ERR', '0', 'S-61001', '86', '84', '5', 'connectionterminated by local host', 'C-11017', 'scanning timeout', '78', 'S-64002', '21', '6', '1', 'standby', '13', '80', '2', 'Q-64002', 'P-41007 ', '79', 'S-65002', 'connection timeout', 'terminate by peer user', 'V-21008', 'active', '14', '8.0'}


In [ ]:
a = list(a)
print(a)

['connection fail to establish', 'NFANDROID2', '4', 'UNKNOWN', '90', '95', 'B-A8002', 'H-51046', 'PHONE_ERR', '0', 'S-61001', '86', '84', '5', 'connectionterminated by local host', 'C-11017', 'scanning timeout', '78', 'S-64002', '21', '6', '1', 'standby', '13', '80', '2', 'Q-64002', 'P-41007 ', '79', 'S-65002', 'connection timeout', 'terminate by peer user', 'V-21008', 'active', '14', '8.0']


In [ ]:
test_err.errcode.unique()

array(['1', '2', 'connection timeout', ..., '5803', '5444', '5691'],
      dtype=object)

In [ ]:
test_err['errcode_pick'] = test_err['errcode'].isin(a).astype('int64')

In [ ]:
test_err.errcode_pick.value_counts()

1    16204694
0      327950
Name: errcode_pick, dtype: int64

In [ ]:
test_err = test_err[test_err.errcode_pick == 1]
test_err

user_id            time model_nm  ... errtype  errcode errcode_pick
0           30000  20201101030227  model_1  ...      31        1            1
1           30000  20201101030227  model_1  ...      33        2            1
2           30000  20201101030228  model_1  ...      15        1            1
3           30000  20201101030256  model_1  ...      22        1            1
4           30000  20201101030300  model_1  ...      11        1            1
...           ...             ...      ...  ...     ...      ...          ...
16532643    44998  20201130210050  model_1  ...      40        0            1
16532644    44998  20201130211831  model_1  ...      31        1            1
16532645    44998  20201130211832  model_1  ...      15        1            1
16532646    44998  20201130212259  model_1  ...      16        1            1
16532647    44998  20201130212316  model_1  ...      31        0            1

[16204694 rows x 7 columns]

In [ ]:
labels = encoder.transform(test_err.errcode)
labels.shape

(16204694,)

In [ ]:
test_err['errcode_label'] = labels

In [ ]:
errcode_pick = [2, 4, 279, 2728, 196, 2796, 2755, 
                2799, 2803, 2771, 2517, 1209, 2500, 176, 
                2798, 2518, 2804, 2768, 2791, 297, 2802, 
                2492, 2776, 2785, 2800,2598, 2579, 2744, 
                2763, 2639, 2774, 2758, 2729, 1537, 2153, 2676]

In [ ]:
test_err['errcode_pick'] = test_err['errcode_label'].isin(errcode_pick).astype('int64')

In [ ]:
test_err = test_err[test_err.errcode_pick == 1]
test_err

user_id            time model_nm  ... errcode  errcode_pick errcode_label
0           30000  20201101030227  model_1  ...       1             1             4
1           30000  20201101030227  model_1  ...       2             1           279
2           30000  20201101030228  model_1  ...       1             1             4
3           30000  20201101030256  model_1  ...       1             1             4
4           30000  20201101030300  model_1  ...       1             1             4
...           ...             ...      ...  ...     ...           ...           ...
16532643    44998  20201130210050  model_1  ...       0             1             2
16532644    44998  20201130211831  model_1  ...       1             1             4
16532645    44998  20201130211832  model_1  ...       1             1             4
16532646    44998  20201130212259  model_1  ...       1             1             4
16532647    44998  20201130212316  model_1  ...       0             1             2

[16204694 rows x 8 columns]

In [ ]:
test_err.errcode_label.value_counts().sort_index()

In [ ]:
labels = encoder2.transform(test_err.errcode_label)
labels.shape

(16204694,)

In [ ]:
test_err['errcode_label_2'] = labels

In [ ]:
test_err.errcode_label_2.value_counts().sort_index()

0     2565532
1     8750154
2       11740
3      316110
4      155401
5         997
6       19084
7       12991
8       13030
9        5059
10     324186
11       9427
12     326179
13     140642
14       3164
15      58820
16      17806
17     855747
18        211
19        113
20     108244
21        142
22        181
23      11808
24      35026
25        136
26       1636
27       3571
28       1064
29     225519
30     109690
31    1890632
32      84809
33        347
34     139947
35       5549
Name: errcode_label_2, dtype: int64

In [ ]:
# a = set(test_err.fwver).intersection(train_err.fwver)
# print(a)

In [ ]:
# a = list(a)
# print(a)

In [ ]:
# test_err['fwver_pick'] = test_err['fwver'].isin(a).astype('int64')

In [ ]:
# test_err = test_err[test_err.fwver_pick == 1]
# test_err

In [ ]:
test_err['code_fwver'] = test_err["fwver"].map(str) + " " + test_err["errcode_label_2"].map(str)

In [ ]:
a = set(test_err.code_fwver).intersection(train_err.code_fwver)
print(a)

{'04.33.1261 17', '03.11.1141 0', '05.15.2114 1', '04.33.1261 11', '04.16.3553 9', '03.11.1167 25', '04.22.1684 2', '04.16.3571 6', '04.16.3571 11', '03.11.1167 26', '04.22.1684 17', '04.16.3571 3', '04.16.3553 22', '05.15.2138 18', '04.22.1778 28', '04.33.1185 3', '04.22.1778 0', '04.33.1261 19', '05.15.2114 0', '05.15.2138 22', '04.33.1149 31', '04.33.1185 6', '04.16.3553 26', '04.22.1750 10', '04.82.1684 9', '04.16.3553 12', '03.11.1167 4', '04.22.1750 35', '04.33.1185 9', '04.33.1149 32', '04.16.3571 10', '8.5.3 24', '04.22.1684 1', '04.22.1750 28', '04.22.1778 13', '04.82.1778 1', '04.22.1750 12', '04.82.1778 6', '04.33.1185 12', '04.33.1149 8', '04.22.1656 26', '04.33.1185 8', '04.82.1778 34', '04.33.1261 13', '04.16.3553 31', '04.33.1185 28', '04.16.3553 27', '05.66.3571 10', '04.82.1730 31', '04.73.2237 0', '03.11.1149 0', '04.22.1778 16', '05.15.2138 4', '04.33.1185 0', '04.16.3571 15', '04.16.3569 12', '05.66.3237 6', '04.33.1185 1', '04.33.1261 26', '04.22.1750 30', '04.33.1

In [ ]:
a = list(a)
print(a)

['04.33.1261 17', '03.11.1141 0', '05.15.2114 1', '04.33.1261 11', '04.16.3553 9', '03.11.1167 25', '04.22.1684 2', '04.16.3571 6', '04.16.3571 11', '03.11.1167 26', '04.22.1684 17', '04.16.3571 3', '04.16.3553 22', '05.15.2138 18', '04.22.1778 28', '04.33.1185 3', '04.22.1778 0', '04.33.1261 19', '05.15.2114 0', '05.15.2138 22', '04.33.1149 31', '04.33.1185 6', '04.16.3553 26', '04.22.1750 10', '04.82.1684 9', '04.16.3553 12', '03.11.1167 4', '04.22.1750 35', '04.33.1185 9', '04.33.1149 32', '04.16.3571 10', '8.5.3 24', '04.22.1684 1', '04.22.1750 28', '04.22.1778 13', '04.82.1778 1', '04.22.1750 12', '04.82.1778 6', '04.33.1185 12', '04.33.1149 8', '04.22.1656 26', '04.33.1185 8', '04.82.1778 34', '04.33.1261 13', '04.16.3553 31', '04.33.1185 28', '04.16.3553 27', '05.66.3571 10', '04.82.1730 31', '04.73.2237 0', '03.11.1149 0', '04.22.1778 16', '05.15.2138 4', '04.33.1185 0', '04.16.3571 15', '04.16.3569 12', '05.66.3237 6', '04.33.1185 1', '04.33.1261 26', '04.22.1750 30', '04.33.1

In [ ]:
test_err['code_fwver_pick'] = test_err['code_fwver'].isin(a).astype('int64')

In [ ]:
test_err = test_err[test_err.code_fwver_pick == 1]
test_err

user_id            time  ...    code_fwver code_fwver_pick
0           30000  20201101030227  ...  04.16.3553 1               1
1           30000  20201101030227  ...  04.16.3553 4               1
2           30000  20201101030228  ...  04.16.3553 1               1
3           30000  20201101030256  ...  04.16.3553 1               1
4           30000  20201101030300  ...  04.16.3553 1               1
...           ...             ...  ...           ...             ...
16532643    44998  20201130210050  ...  04.16.3553 0               1
16532644    44998  20201130211831  ...  04.16.3553 1               1
16532645    44998  20201130211832  ...  04.16.3553 1               1
16532646    44998  20201130212259  ...  04.16.3553 1               1
16532647    44998  20201130212316  ...  04.16.3553 0               1

[16197938 rows x 11 columns]

In [ ]:
labels = encoder_fw.transform(test_err.code_fwver)
labels.shape

(16197938,)

In [ ]:
test_err['code_fwver_label'] = labels

In [ ]:
test_err.code_fwver_label.value_counts().sort_index()

0        31
1      1537
3         1
4       522
5      4030
       ... 
420      20
421     516
423      45
425      10
426       3
Name: code_fwver_label, Length: 359, dtype: int64

In [ ]:
id_code_fw = test_err[['user_id','code_fwver_label']].values
test_code_fw = np.zeros((test_user_number,428))

for person_idx, code_fw in tqdm(id_code_fw):
    # person_idx - test_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    test_code_fw[person_idx - test_user_id_min, code_fw] += 1
test_code_fw = test_code_fw.reshape(test_code_fw.shape[0],-1)
print(test_code_fw.shape)

100%|██████████| 16197938/16197938 [00:42<00:00, 379161.21it/s]

(14999, 428)


In [ ]:
test_type_code_fw = np.append(test_x, test_code_fw, axis=1)
test_type_code_fw.shape

(14999, 470)

In [ ]:
# labels = encoder_fw.transform(test_err.fwver)
# labels.shape

In [ ]:
# test_err['fwver_label'] = labels

In [ ]:
# test_err.fwver_label.value_counts().sort_index()

In [ ]:
# len(test_err.fwver_label.unique())

In [ ]:
# id_fwver_test = test_err[['user_id','fwver_label']].values
# test_fwver = np.zeros((test_user_number,37))

# for person_idx, fwv in tqdm(id_fwver_test):
#     # person_idx - test_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
#     test_fwver[person_idx - test_user_id_min, fwv] += 1
# test_fwver = test_fwver.reshape(test_fwver.shape[0],-1)
# print(test_fwver.shape)

In [ ]:
# test_type_code_fwver = np.append(test_type_code, test_fwver, axis=1)
# test_type_code_fwver.shape

In [ ]:
# 예측
pred_y_list = []
for model in models:
    pred_y = model.predict(test_type_code_fw)
    pred_y_list.append(pred_y.reshape(-1,1))
    
pred_ensemble = np.mean(pred_y_list, axis = 0)

In [ ]:
pred_ensemble

array([[0.8250856 ],
       [0.20382505],
       [0.58738559],
       ...,
       [0.71115179],
       [0.8321468 ],
       [0.35991095]])

In [ ]:
sample_submssion = pd.read_csv(PATH+'sample_submission.csv')

In [ ]:
sample_submssion['problem'] = pred_ensemble.reshape(-1)

In [ ]:
sample_submssion.to_csv("dacon_baseline_errcode_type_code_pick_fwver.csv", index = False)
sample_submssion

user_id   problem
0        30000  0.825086
1        30001  0.203825
2        30002  0.587386
3        30003  0.774694
4        30004  0.771344
...        ...       ...
14994    44994  0.218175
14995    44995  0.333693
14996    44996  0.711152
14997    44997  0.832147
14998    44998  0.359911

[14999 rows x 2 columns]